# 簡單線性迴歸 CRISP-DM 完整分析

本筆記本展示如何使用 CRISP-DM 方法論來實作簡單線性迴歸模型。

## CRISP-DM 六個階段：
1. **Business Understanding** - 業務理解
2. **Data Understanding** - 資料理解
3. **Data Preparation** - 資料準備
4. **Modeling** - 建模
5. **Evaluation** - 評估
6. **Deployment** - 部署

## 1. 匯入必要的函式庫

In [ ]:
# 資料處理和數值計算
import numpy as np
import pandas as pd

# 視覺化
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# 機器學習
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# 互動式工具
from ipywidgets import interact, widgets
import ipywidgets as widgets
from IPython.display import display

# 設定
import warnings
warnings.filterwarnings('ignore')

# 設定中文字體
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 設定圖表樣式
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ 所有函式庫匯入成功！")

## 2. 業務理解：定義問題與需求

### 問題定義：
- 建立簡單線性迴歸模型來理解兩個變數之間的線性關係
- 驗證模型能否正確識別真實的線性參數
- 評估噪音對模型效能的影響

### 輸入參數：
- **斜率 (a)**: 真實的線性關係斜率
- **截距 (b)**: 真實的線性關係截距
- **噪音標準差**: 資料中隨機噪音的大小
- **資料點數量**: 訓練資料的樣本數

### 預期輸出：
- 訓練好的線性迴歸模型
- 模型效能評估指標 (R², MSE, RMSE)
- 參數比較 (真實 vs 預測)
- 視覺化結果

In [ ]:
# 定義業務需求和成功標準
business_requirements = {
    "目標": "建立準確的簡單線性迴歸模型",
    "成功標準": {
        "R²": "> 0.8 (解釋80%以上的變異)",
        "參數準確性": "斜率和截距誤差 < 10%",
        "穩健性": "在不同噪音水平下保持良好效能"
    },
    "應用場景": [
        "銷售額與廣告支出關係分析",
        "房價與面積關係預測",
        "溫度與能源消耗關係建模",
        "學習時間與考試成績關係探索"
    ]
}

print("📋 業務需求分析：")
for key, value in business_requirements.items():
    print(f"\n{key}: {value}")

## 3. 資料理解：生成合成資料

我們將生成符合以下公式的合成資料：
$$y = ax + b + \epsilon$$

其中：
- $a$ 是斜率
- $b$ 是截距
- $\epsilon$ 是噪音項，服從正態分布 $N(0, \sigma^2)$

In [ ]:
def generate_linear_data(slope=2.5, intercept=1.0, noise_std=0.5, n_points=100, x_range=(-5, 5), random_seed=42):
    """
    生成線性迴歸的合成資料
    
    參數:
    - slope: 斜率
    - intercept: 截距
    - noise_std: 噪音標準差
    - n_points: 資料點數量
    - x_range: x值範圍
    - random_seed: 隨機種子
    
    回傳:
    - DataFrame: 包含 x, y, y_true, noise 的資料
    """
    np.random.seed(random_seed)
    
    # 生成 x 值
    x = np.linspace(x_range[0], x_range[1], n_points)
    
    # 計算真實的 y 值（無噪音）
    y_true = slope * x + intercept
    
    # 生成噪音
    noise = np.random.normal(0, noise_std, n_points)
    
    # 加上噪音得到觀測值
    y = y_true + noise
    
    # 建立 DataFrame
    data = pd.DataFrame({
        'x': x,
        'y': y,
        'y_true': y_true,
        'noise': noise
    })
    
    return data

# 生成預設資料
default_params = {
    'slope': 2.5,
    'intercept': 1.0,
    'noise_std': 0.5,
    'n_points': 100
}

data = generate_linear_data(**default_params)

print(f"✅ 資料生成完成！")
print(f"資料形狀: {data.shape}")
print(f"\n前5筆資料:")
display(data.head())

## 4. 資料視覺化與探索

In [ ]:
# 基本統計描述
print("📊 資料統計描述:")
display(data.describe())

# 相關係數
correlation = data['x'].corr(data['y'])
print(f"\n📈 x 和 y 的相關係數: {correlation:.4f}")

In [ ]:
# 建立多面板視覺化
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. 散點圖與真實關係
axes[0, 0].scatter(data['x'], data['y'], alpha=0.6, color='blue', s=30, label='觀測資料')
axes[0, 0].plot(data['x'], data['y_true'], 'r-', linewidth=3, label=f'真實關係 (y = {default_params["slope"]}x + {default_params["intercept"]})')
axes[0, 0].set_xlabel('x')
axes[0, 0].set_ylabel('y')
axes[0, 0].set_title('原始資料散點圖')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. x 的分布
axes[0, 1].hist(data['x'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 1].set_xlabel('x')
axes[0, 1].set_ylabel('頻率')
axes[0, 1].set_title('x 變數分布')
axes[0, 1].grid(True, alpha=0.3)

# 3. y 的分布
axes[1, 0].hist(data['y'], bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1, 0].set_xlabel('y')
axes[1, 0].set_ylabel('頻率')
axes[1, 0].set_title('y 變數分布')
axes[1, 0].grid(True, alpha=0.3)

# 4. 噪音分布
axes[1, 1].hist(data['noise'], bins=20, alpha=0.7, color='orange', edgecolor='black')
axes[1, 1].set_xlabel('噪音')
axes[1, 1].set_ylabel('頻率')
axes[1, 1].set_title('噪音分布')
axes[1, 1].grid(True, alpha=0.3)

# 在噪音分布圖上加上理論正態分布
noise_x = np.linspace(data['noise'].min(), data['noise'].max(), 100)
noise_theoretical = (len(data) * (noise_x[1] - noise_x[0]) * 
                    (1 / (default_params['noise_std'] * np.sqrt(2 * np.pi))) * 
                    np.exp(-0.5 * (noise_x / default_params['noise_std'])**2))
axes[1, 1].plot(noise_x, noise_theoretical, 'r-', linewidth=2, label='理論正態分布')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 5. 資料準備：建立DataFrame結構

In [ ]:
def prepare_data(data):
    """
    準備建模所需的資料格式
    """
    # 檢查缺失值
    missing_values = data.isnull().sum()
    print("🔍 資料品質檢查:")
    print(f"缺失值: {missing_values.sum()}")
    
    if missing_values.sum() > 0:
        print("缺失值詳情:")
        print(missing_values[missing_values > 0])
    else:
        print("✅ 無缺失值")
    
    # 準備特徵和目標變數
    X = data[['x']]  # 特徵矩陣（需要是二維）
    y = data['y']    # 目標變數
    
    print(f"\n📐 資料維度:")
    print(f"特徵矩陣 X: {X.shape}")
    print(f"目標變數 y: {y.shape}")
    
    # 資料範圍
    print(f"\n📊 資料範圍:")
    print(f"x: [{X['x'].min():.2f}, {X['x'].max():.2f}]")
    print(f"y: [{y.min():.2f}, {y.max():.2f}]")
    
    return X, y

# 準備資料
X, y = prepare_data(data)
print("\n✅ 資料準備完成！")

## 6. 建模：實作簡單線性迴歸

In [ ]:
def train_linear_regression(X, y, method='sklearn'):
    """
    訓練線性迴歸模型
    
    參數:
    - X: 特徵矩陣
    - y: 目標變數
    - method: 'sklearn' 或 'numpy'
    
    回傳:
    - model: 訓練好的模型
    - predictions: 預測值
    - coefficients: 模型係數
    """
    
    if method == 'sklearn':
        # 使用 scikit-learn
        model = LinearRegression()
        model.fit(X, y)
        
        predictions = model.predict(X)
        slope = model.coef_[0]
        intercept = model.intercept_
        
    elif method == 'numpy':
        # 使用 numpy 最小二乘法
        # 添加偏置項（截距）
        X_with_bias = np.column_stack([np.ones(len(X)), X.values])
        
        # 計算係數 (X^T X)^(-1) X^T y
        coeffs = np.linalg.lstsq(X_with_bias, y, rcond=None)[0]
        intercept = coeffs[0]
        slope = coeffs[1]
        
        predictions = X_with_bias @ coeffs
        
        # 建立類似 sklearn 的模型物件
        class SimpleLinearModel:
            def __init__(self, slope, intercept):
                self.coef_ = [slope]
                self.intercept_ = intercept
            
            def predict(self, X):
                return X.values.flatten() * self.coef_[0] + self.intercept_
        
        model = SimpleLinearModel(slope, intercept)
    
    coefficients = {
        'slope': slope,
        'intercept': intercept
    }
    
    return model, predictions, coefficients

# 使用兩種方法訓練模型
print("🤖 開始訓練線性迴歸模型...\n")

# 方法1: scikit-learn
print("📚 方法1: scikit-learn LinearRegression")
model_sklearn, pred_sklearn, coef_sklearn = train_linear_regression(X, y, 'sklearn')
print(f"斜率: {coef_sklearn['slope']:.4f}")
print(f"截距: {coef_sklearn['intercept']:.4f}")

# 方法2: numpy 最小二乘法
print("\n📊 方法2: numpy 最小二乘法")
model_numpy, pred_numpy, coef_numpy = train_linear_regression(X, y, 'numpy')
print(f"斜率: {coef_numpy['slope']:.4f}")
print(f"截距: {coef_numpy['intercept']:.4f}")

# 比較兩種方法的結果
print("\n🔍 方法比較:")
print(f"斜率差異: {abs(coef_sklearn['slope'] - coef_numpy['slope']):.6f}")
print(f"截距差異: {abs(coef_sklearn['intercept'] - coef_numpy['intercept']):.6f}")

# 使用 sklearn 模型作為主要模型
model = model_sklearn
predictions = pred_sklearn
coefficients = coef_sklearn

print("\n✅ 模型訓練完成！")

In [ ]:
# 比較真實參數與預測參數
print("📊 參數比較分析:")
print("=" * 40)

comparison_data = {
    '參數': ['斜率', '截距'],
    '真實值': [default_params['slope'], default_params['intercept']],
    '預測值': [coefficients['slope'], coefficients['intercept']],
    '絕對誤差': [abs(coefficients['slope'] - default_params['slope']), 
                abs(coefficients['intercept'] - default_params['intercept'])],
    '相對誤差(%)': [abs(coefficients['slope'] - default_params['slope']) / abs(default_params['slope']) * 100,
                  abs(coefficients['intercept'] - default_params['intercept']) / abs(default_params['intercept']) * 100]
}

comparison_df = pd.DataFrame(comparison_data)
display(comparison_df)

## 7. 模型評估：計算評估指標

In [ ]:
def evaluate_model(y_true, y_pred, model_name="Linear Regression"):
    """
    評估模型效能
    
    參數:
    - y_true: 真實值
    - y_pred: 預測值
    - model_name: 模型名稱
    
    回傳:
    - metrics: 評估指標字典
    """
    
    # 計算各種評估指標
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    
    # 計算調整後的 R²
    n = len(y_true)
    p = 1  # 特徵數量
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    
    metrics = {
        'R²': r2,
        '調整後R²': adj_r2,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae
    }
    
    return metrics

# 評估模型
metrics = evaluate_model(y, predictions)

print("📈 模型評估結果:")
print("=" * 30)

for metric_name, value in metrics.items():
    print(f"{metric_name:12}: {value:.4f}")

# R² 解釋
print(f"\n💡 R² 解釋:")
r2_percentage = metrics['R²'] * 100
print(f"模型解釋了 {r2_percentage:.1f}% 的變異")

if metrics['R²'] >= 0.9:
    performance = "非常好 🎉"
elif metrics['R²'] >= 0.7:
    performance = "良好 👍"
elif metrics['R²'] >= 0.5:
    performance = "中等 👌"
else:
    performance = "需要改善 📉"

print(f"模型效能: {performance}")

## 8. 視覺化結果：繪製迴歸線

In [ ]:
# 建立綜合視覺化
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. 主要擬合結果
axes[0, 0].scatter(X['x'], y, alpha=0.6, color='blue', s=40, label='觀測資料')
axes[0, 0].plot(X['x'], predictions, 'r-', linewidth=3, 
                label=f'預測線 (y = {coefficients["slope"]:.2f}x + {coefficients["intercept"]:.2f})')
axes[0, 0].plot(X['x'], data['y_true'], 'g--', linewidth=3, 
                label=f'真實關係 (y = {default_params["slope"]}x + {default_params["intercept"]})')
axes[0, 0].set_xlabel('x')
axes[0, 0].set_ylabel('y')
axes[0, 0].set_title(f'線性迴歸結果 (R² = {metrics["R²"]:.3f})')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 殘差圖
residuals = y - predictions
axes[0, 1].scatter(predictions, residuals, alpha=0.6, color='purple', s=40)
axes[0, 1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('預測值')
axes[0, 1].set_ylabel('殘差')
axes[0, 1].set_title('殘差圖')
axes[0, 1].grid(True, alpha=0.3)

# 3. 預測 vs 實際
axes[1, 0].scatter(y, predictions, alpha=0.6, color='green', s=40)
min_val = min(y.min(), predictions.min())
max_val = max(y.max(), predictions.max())
axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='完美預測線')
axes[1, 0].set_xlabel('實際值')
axes[1, 0].set_ylabel('預測值')
axes[1, 0].set_title('預測 vs 實際值')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. 殘差分布
axes[1, 1].hist(residuals, bins=20, alpha=0.7, color='orange', edgecolor='black', density=True)
axes[1, 1].set_xlabel('殘差')
axes[1, 1].set_ylabel('機率密度')
axes[1, 1].set_title('殘差分布')

# 在殘差分布上疊加正態分布
residual_std = np.std(residuals)
residual_mean = np.mean(residuals)
x_norm = np.linspace(residuals.min(), residuals.max(), 100)
y_norm = (1 / (residual_std * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x_norm - residual_mean) / residual_std)**2)
axes[1, 1].plot(x_norm, y_norm, 'r-', linewidth=2, label='理論正態分布')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 額外的統計檢定
print("🔍 殘差分析:")
print(f"殘差平均值: {residuals.mean():.4f} (應接近0)")
print(f"殘差標準差: {residuals.std():.4f}")
print(f"殘差偏度: {residuals.skew():.4f} (應接近0)")
print(f"殘差峰度: {residuals.kurtosis():.4f} (應接近0)")

## 9. 互動式參數探索

使用互動式工具來探索不同參數對模型效能的影響

In [ ]:
def interactive_linear_regression(slope=2.5, intercept=1.0, noise_std=0.5, n_points=100):
    """
    互動式線性迴歸演示
    """
    # 生成資料
    data_interactive = generate_linear_data(slope, intercept, noise_std, n_points)
    
    # 準備資料
    X_interactive = data_interactive[['x']]
    y_interactive = data_interactive['y']
    
    # 訓練模型
    model_interactive = LinearRegression()
    model_interactive.fit(X_interactive, y_interactive)
    pred_interactive = model_interactive.predict(X_interactive)
    
    # 計算指標
    r2_interactive = r2_score(y_interactive, pred_interactive)
    mse_interactive = mean_squared_error(y_interactive, pred_interactive)
    
    # 參數誤差
    slope_error = abs(model_interactive.coef_[0] - slope)
    intercept_error = abs(model_interactive.intercept_ - intercept)
    
    # 視覺化
    plt.figure(figsize=(12, 8))
    
    # 主圖
    plt.subplot(2, 2, 1)
    plt.scatter(X_interactive['x'], y_interactive, alpha=0.6, color='blue', s=30)
    plt.plot(X_interactive['x'], pred_interactive, 'r-', linewidth=3, 
             label=f'預測: y = {model_interactive.coef_[0]:.2f}x + {model_interactive.intercept_:.2f}')
    plt.plot(X_interactive['x'], data_interactive['y_true'], 'g--', linewidth=2, 
             label=f'真實: y = {slope}x + {intercept}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'R² = {r2_interactive:.3f}, MSE = {mse_interactive:.3f}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 殘差圖
    plt.subplot(2, 2, 2)
    residuals_interactive = y_interactive - pred_interactive
    plt.scatter(pred_interactive, residuals_interactive, alpha=0.6, color='purple')
    plt.axhline(y=0, color='red', linestyle='--')
    plt.xlabel('預測值')
    plt.ylabel('殘差')
    plt.title('殘差圖')
    plt.grid(True, alpha=0.3)
    
    # 參數比較
    plt.subplot(2, 2, 3)
    params = ['斜率', '截距']
    true_vals = [slope, intercept]
    pred_vals = [model_interactive.coef_[0], model_interactive.intercept_]
    
    x_pos = np.arange(len(params))
    width = 0.35
    
    plt.bar(x_pos - width/2, true_vals, width, label='真實值', alpha=0.7)
    plt.bar(x_pos + width/2, pred_vals, width, label='預測值', alpha=0.7)
    
    plt.xlabel('參數')
    plt.ylabel('值')
    plt.title('參數比較')
    plt.xticks(x_pos, params)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 誤差分析
    plt.subplot(2, 2, 4)
    errors = [slope_error, intercept_error]
    plt.bar(params, errors, alpha=0.7, color='red')
    plt.xlabel('參數')
    plt.ylabel('絕對誤差')
    plt.title('參數預測誤差')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 輸出數值結果
    print(f"📊 模型效能: R² = {r2_interactive:.4f}, MSE = {mse_interactive:.4f}")
    print(f"📐 參數誤差: 斜率 = {slope_error:.4f}, 截距 = {intercept_error:.4f}")

# 建立互動式介面
interactive_plot = widgets.interactive(
    interactive_linear_regression,
    slope=widgets.FloatSlider(value=2.5, min=-5.0, max=5.0, step=0.1, description='斜率:'),
    intercept=widgets.FloatSlider(value=1.0, min=-5.0, max=5.0, step=0.1, description='截距:'),
    noise_std=widgets.FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description='噪音:'),
    n_points=widgets.IntSlider(value=100, min=50, max=300, step=10, description='點數:')
)

display(interactive_plot)

## 10. 部署：建立Streamlit應用程式

以下程式碼展示如何建立一個簡單的 Streamlit 應用程式。將此程式碼儲存為獨立的 `.py` 檔案並執行。

In [ ]:
# Streamlit 應用程式程式碼（供參考）
streamlit_code = '''
import streamlit as st
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

st.title("📈 簡單線性迴歸 CRISP-DM 演示")

# 側邊欄參數
st.sidebar.header("參數設定")
slope = st.sidebar.slider("斜率", -5.0, 5.0, 2.5, 0.1)
intercept = st.sidebar.slider("截距", -5.0, 5.0, 1.0, 0.1)
noise_std = st.sidebar.slider("噪音標準差", 0.0, 2.0, 0.5, 0.1)
n_points = st.sidebar.slider("資料點數量", 50, 300, 100, 10)

# 生成資料和訓練模型的函數
def generate_and_model(slope, intercept, noise_std, n_points):
    np.random.seed(42)
    x = np.linspace(-5, 5, n_points)
    y_true = slope * x + intercept
    noise = np.random.normal(0, noise_std, n_points)
    y = y_true + noise
    
    X = x.reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)
    
    return x, y, y_true, y_pred, model

# 執行分析
x, y, y_true, y_pred, model = generate_and_model(slope, intercept, noise_std, n_points)

# 計算指標
r2 = r2_score(y, y_pred)
mse = mean_squared_error(y, y_pred)

# 顯示結果
col1, col2, col3 = st.columns(3)
col1.metric("R²", f"{r2:.3f}")
col2.metric("MSE", f"{mse:.3f}")
col3.metric("點數", n_points)

# 繪圖
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='觀測資料'))
fig.add_trace(go.Scatter(x=x, y=y_true, mode='lines', name='真實關係'))
fig.add_trace(go.Scatter(x=x, y=y_pred, mode='lines', name='預測線'))
fig.update_layout(title='線性迴歸結果', xaxis_title='x', yaxis_title='y')

st.plotly_chart(fig, use_container_width=True)
'''

print("📋 Streamlit 應用程式程式碼:")
print("將上方程式碼儲存為 'streamlit_app.py' 並執行: streamlit run streamlit_app.py")
print("\n💡 部署選項:")
print("1. 本地執行: streamlit run streamlit_app.py")
print("2. Streamlit Cloud: 上傳到 GitHub 並連接 Streamlit Cloud")
print("3. Heroku: 建立 Heroku 應用程式")
print("4. Docker: 建立容器化應用程式")

## 總結

本筆記本完整展示了使用 CRISP-DM 方法論實作簡單線性迴歸的全過程：

### ✅ 完成項目：

1. **Business Understanding**: 定義了問題、成功標準和應用場景
2. **Data Understanding**: 生成合成資料並進行探索性分析
3. **Data Preparation**: 準備建模所需的資料格式
4. **Modeling**: 使用 scikit-learn 和 numpy 兩種方法建模
5. **Evaluation**: 計算多種評估指標並進行視覺化分析
6. **Deployment**: 提供 Streamlit 應用程式部署方案

### 📊 主要發現：

- 模型能夠準確識別線性關係的參數
- 噪音水平直接影響模型效能
- 殘差分析顯示模型假設的合理性
- 互動式工具有助於參數探索

### 🚀 後續步驟：

1. 部署 Streamlit 應用程式
2. 擴展到多元線性迴歸
3. 加入交叉驗證
4. 探索非線性關係建模